**네이버 영화 페이지에서 댓글 크롤링하기**  

## 라이브러리 및 페이지 불러오기

In [ ]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup as bs

In [ ]:
# 최초 화면을 불러온다.
# 영화 코드 입력하기

#  미션 임파서블: 로그네이션

movie_code = 95541
url=f"https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={movie_code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false"    
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}
response=requests.get(url,headers=headers) 
html = bs(response.text,'lxml')

## 전체 댓글 수 및 전체 페이지 수 확인

In [ ]:
# 크롤링한 데이터에서 전체 댓글 수를 나타낸 부분을 가져온다
total = html.select('div.score_total > strong.total > em')
cnt = int(total[0].text.replace(',',''))

# 각 페이지마다 댓글이 10개씩 있으므로
# 댓글수(cnt)가 10의 배수이면 전체 페이지는 나누기 10의 값이고
# 10의 배수가 아니면 나누기 10을 한 값에 1페이지를 더해준다.
page_end = int(cnt / 10)
if cnt % 10 != 0:
    page_end += 1

In [ ]:
page_end

1385

## get_review_list 함수 정의

In [ ]:
# 페이지 번호, 댓글 리스트를 매개변수로 설정하면
# 순차적으로 댓글과 여러 정보를 딕셔너리로 저장 후 리스트에 최종으로 담는다. (별다른 리턴값은 없다)
import re

def get_review_list(page_no, review_list, movie_code):
    """
    페이지별로 10개씩 댓글 및 평점을 수집하여 리스트에 넣는다.
    """
    url=f"https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={movie_code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page_no}"    
    headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}
    response=requests.get(url,headers=headers) 
    html = bs(response.text,'lxml')
    
    # 크롤링한 데이터에서 댓글(review)과 평점(score) 부분만 불러와서 리스트에 넣는 코드    
    review = html.select('div.score_result > ul > li > div.score_reple > p > span')
    score = html.select('div.score_result > ul > li > div.star_score > em')

    # review에는 관람 여부, 스포일러 여부, 댓글 총 3가지의 데이터가 있다.
    # 상황에 따라 알맞는 데이터를 가져오기 위해 제어변수 num을 설정 및 초기화 하고,
    # 댓글 자체의 순서를 제어할 cnt 변수 초기화한다.
    num = 0
    cnt = 0
    
    # 공감 비공감 전체 리스트
    like = html.select('div.btn_area > a._sympathyButton> strong')
    unlike = html.select('div.btn_area > a._notSympathyButton > strong')  
    
    while (num < len(review)):
        # 정보를 담을 딕셔너리 초기화
        info_dic = {"댓글번호":'', "작성일자":'', "댓글":'', "평점":0, "관람여부":0, "스포여부":0, "공감수":0, "비공감수":0, "id":''} 

        # 댓글번호 추출
        review_no = str(html.select('div.score_reple > dl > dt > em > a')[cnt])
        info_dic["댓글번호"] = int(re.findall('\d+',str(review_no))[0])
        
        # id 및 작성일자
        info_dic["id"] = html.select('div.score_reple > dl > dt ')[cnt].text.strip().split('\n\n\n')[0]
        info_dic["작성일자"] = html.select('div.score_reple > dl > dt ')[cnt].text.strip().split('\n\n\n')[1]
        # temp2=html.select('div.score_reple > dl > dt ')[0].text.strip().split('\n\n\n') #여기서 0이 for문에서 바뀔숫자
        
        # 댓글 추출 (관람 여부, 스포일러 여부 추가)
        # 관람 여부 확인 -> 스포일러 여부 확인 -> 댓글 저장 순으로 진행한다.
        # 댓글이 저장되면 최종 저장한 인덱스 그 다음으로 num을 이동시킨다.
        if review[num].text == '관람객':
            info_dic["관람여부"] = 1
            if review[num+1].text == '스포일러가 포함된 감상평입니다. 감상평 보기':
                info_dic["스포여부"] = 1
                info_dic["댓글"] = review[num+2].text.strip()
                num += 3
            else :   
                info_dic["댓글"] = review[num+1].text.strip()
                num += 2
        elif review[num].text == '스포일러가 포함된 감상평입니다. 감상평 보기':
            info_dic["스포여부"] = 1
            info_dic["댓글"] = review[num+1].text.strip()
            num += 2
        else :
            info_dic["댓글"] = review[num].text.strip()
            num += 1

        # 평점 추출
        info_dic["평점"] = score[cnt].text.strip()
        
        # 공감, 비공감수 추출
        info_dic["공감수"] = int(re.findall('\d+',str(like[cnt]))[1])
        info_dic["비공감수"] = int(re.findall('\d+',str(unlike[cnt]))[1])            
        
        # 댓글의 모든 정보를 넣었으니 순서를 다음으로 넘긴다.
        cnt += 1
        # 정보가 담긴 딕셔너리를 리스트에 저장
        review_list.append(info_dic)


## 반복을 통한 전체 댓글 및 평점 저장

In [ ]:
# 댓글 리스트, 평점 리스트, 페이지 번호 초기화
review_list = []
page_no = 1

In [ ]:
# # 처음부터 마지막 페이지까지 반복

while (page_no <= page_end):
    # get_review_list 함수를 통해 리뷰와 평점을 각각의 리스트에 담는다
    get_review_list(page_no, review_list, movie_code)
    page_no += 1

## 리뷰와 평점 병합해주기 

In [ ]:
review_df = pd.DataFrame(review_list)
review_df

,댓글번호,작성일자,댓글,평점,관람여부,스포여부,공감수,비공감수,id
0,10159722,2015.07.30 04:30,우리형 클래스 아시져?? 저 나이에 대역없이 실제 비행기에 매달려 하늘 나실 정도...,10,0,0,3188,238,언제나늘(yoon****)
1,10159167,2015.07.30 00:31,톰크루즈는 늙지를 않는구나...,10,0,0,1917,138,눈의 꽃(mona****)
2,10159630,2015.07.30 03:12,심야보고 왔는데 아빠 뻘인 톰 크루즈 님의 위엄을 보고 오늘 새벽 잠을 못 이루겠습니다.,10,0,0,1759,135,mins****
3,10159035,2015.07.30 00:01,정말 시원하고 재밌어요~,10,0,0,1355,118,dlrj****
4,10159449,2015.07.30 01:48,톰 삼춘 역시 의리 입니다 미션임파서블5 영화도 대박입니다 정말 실망시키지 않는 액...,10,0,0,1292,103,맑은서울(leei****)
...,...,...,...,...,...,...,...,...,...
13836,10159923,2015.07.30 08:43,무슨또6편제작씩이나..그닥별로..미니언즈보러간다,1,0,0,9,37,lee7****
13837,10159597,2015.07.30 03:00,흠.... 이 글들 다 알바생들인가,1,0,0,12,48,유후(daye****)
13838,10159154,2015.07.30 00:29,킹스맨 분노의질주보다는 약하네요,4,0,0,10,47,프레이밍(rort****)
13839,10159835,2015.07.30 07:41,암살화이팅.ㅋㅋㅋ.ㅎㅎㅎ,1,0,0,9,47,lws7****


In [ ]:
review_df["영화제목"] = "미션_임파서블_로그네이션"
review_df

,댓글번호,작성일자,댓글,평점,관람여부,스포여부,공감수,비공감수,id,영화제목
0,10159722,2015.07.30 04:30,우리형 클래스 아시져?? 저 나이에 대역없이 실제 비행기에 매달려 하늘 나실 정도...,10,0,0,3188,238,언제나늘(yoon****),미션_임파서블_로그네이션
1,10159167,2015.07.30 00:31,톰크루즈는 늙지를 않는구나...,10,0,0,1917,138,눈의 꽃(mona****),미션_임파서블_로그네이션
2,10159630,2015.07.30 03:12,심야보고 왔는데 아빠 뻘인 톰 크루즈 님의 위엄을 보고 오늘 새벽 잠을 못 이루겠습니다.,10,0,0,1759,135,mins****,미션_임파서블_로그네이션
3,10159035,2015.07.30 00:01,정말 시원하고 재밌어요~,10,0,0,1355,118,dlrj****,미션_임파서블_로그네이션
4,10159449,2015.07.30 01:48,톰 삼춘 역시 의리 입니다 미션임파서블5 영화도 대박입니다 정말 실망시키지 않는 액...,10,0,0,1292,103,맑은서울(leei****),미션_임파서블_로그네이션
...,...,...,...,...,...,...,...,...,...,...
13836,10159923,2015.07.30 08:43,무슨또6편제작씩이나..그닥별로..미니언즈보러간다,1,0,0,9,37,lee7****,미션_임파서블_로그네이션
13837,10159597,2015.07.30 03:00,흠.... 이 글들 다 알바생들인가,1,0,0,12,48,유후(daye****),미션_임파서블_로그네이션
13838,10159154,2015.07.30 00:29,킹스맨 분노의질주보다는 약하네요,4,0,0,10,47,프레이밍(rort****),미션_임파서블_로그네이션
13839,10159835,2015.07.30 07:41,암살화이팅.ㅋㅋㅋ.ㅎㅎㅎ,1,0,0,9,47,lws7****,미션_임파서블_로그네이션


In [ ]:
# review_df["작성일자"].min

In [ ]:
review_df.to_csv("2015_미션_임파서블_로그네이션.csv", index=False)

In [ ]:
e = pd.read_csv("2015_미션_임파서블_로그네이션.csv")
e.head()

,댓글번호,작성일자,댓글,평점,관람여부,스포여부,공감수,비공감수,id,영화제목
0,10159722,2015.07.30 04:30,우리형 클래스 아시져?? 저 나이에 대역없이 실제 비행기에 매달려 하늘 나실 정도...,10,0,0,3188,238,언제나늘(yoon****),미션_임파서블_로그네이션
1,10159167,2015.07.30 00:31,톰크루즈는 늙지를 않는구나...,10,0,0,1917,138,눈의 꽃(mona****),미션_임파서블_로그네이션
2,10159630,2015.07.30 03:12,심야보고 왔는데 아빠 뻘인 톰 크루즈 님의 위엄을 보고 오늘 새벽 잠을 못 이루겠습니다.,10,0,0,1759,135,mins****,미션_임파서블_로그네이션
3,10159035,2015.07.30 00:01,정말 시원하고 재밌어요~,10,0,0,1355,118,dlrj****,미션_임파서블_로그네이션
4,10159449,2015.07.30 01:48,톰 삼춘 역시 의리 입니다 미션임파서블5 영화도 대박입니다 정말 실망시키지 않는 액...,10,0,0,1292,103,맑은서울(leei****),미션_임파서블_로그네이션
